# Proposal 4

$$
REWRTIE
$$

In [ ]:
%load_ext autoreload
%autoreload 2

# Libraries

In [ ]:
import pickle
import pandas as pd
import plotly.express as px
from tqdm import tqdm

In [ ]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    confusion_matrix_metrics,
    forgetting_metrics,
    timing_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    disk_usage_metrics,
    ram_usage_metrics,
    loss_metrics,
    class_accuracy_metrics,
    accuracy_matrix_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD

## Custom Libraries

In [ ]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.training import ESEWC
from base_code.plugins import WeightStoragePlugin

# Device

In [ ]:
from torch import cuda
device = "cuda" if cuda.is_available() else "cpu"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [ ]:
scenario = PermutedMNIST(10, seed=1234, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [ ]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

## Evaluation metrics definition

In [ ]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    
    loggers=[InteractiveLogger()]
)

## Plugin defitinitions

In [ ]:
# model_plugins = [WeightStoragePlugin()]

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [ ]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(device)
optimizer = SGD(model.parameters(), lr=1e-3)
criterion = CrossEntropyLoss().to(device)
strategy = ESEWC(
    model,
    optimizer,
    criterion,
    ewc_lambda=0.5,
    train_epochs=1,
    train_mb_size=128,
    # plugins=model_plugins,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=device,
    model_arch={
        "num_classes": scenario.n_classes,
        "input_size": 28 * 28,
        "hidden_layers": 2,
        "hidden_size": 100
    }
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [ ]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

# Get metrics

In [ ]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(10):
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp00{i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp00{i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp00{i}"
    ][1]
    # class_accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
    #     f"Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/{i}"
    # ][1]
    # losses[f"Task{i}"] = eval_plugin.get_all_metrics()[
    #     f"Loss_Exp/eval_phase/test_stream/Task000/Exp00{i}"
    # ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

In [ ]:
# train_cpu_usage = eval_plugin.get_all_metrics()["CPUUsage_MB/train_phase/train_stream/Task000"][1]
# train_time_epoch = eval_plugin.get_all_metrics()["Time_Epoch/train_phase/train_stream/Task000"][1]
# train_running_time = eval_plugin.get_all_metrics()["RunningTime_Epoch/train_phase/train_stream/Task000"][1]
# train_loss_epoch = eval_plugin.get_all_metrics()["Loss_MB/train_phase/train_stream/Task000"][1]

In [ ]:
# all_loss_components = strategy.get_store_loss()

In [ ]:
#weights = model_plugins[0].weights

# Store metrics

In [ ]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
    # "class_accuracies": class_accuracies,
    # "losses": losses,
    # "train_cpu_usage": train_cpu_usage,
    # "train_time_epoch": train_time_epoch,
    # "train_running_time": train_running_time,
    # "train_loss_epoch": train_loss_epoch,
    # "all_loss_components": all_loss_components,
    # "weights": weights
}, open(SAVED_METRICS_PATH / "cewc_v1.pkl", "wb"))

# Load

In [ ]:
metrics = pickle.load(open(SAVED_METRICS_PATH / "cewc_v1.pkl", "rb"))

# Plot metrics

## Accuracy

In [ ]:
fig = px.line(
    pd.DataFrame(metrics["accuracies"], index=range(1, len(metrics["accuracies"]["Overall"]) + 1)),
    title="Accuracy vs Experience",
    labels=dict(index="Experience", value="Accuracy"),
    width=800,
    height=600,
    range_y=[0, 1],
    markers=True,
)
fig.update_xaxes(type="category")
fig.show()

## Forgetting

In [ ]:
forgetting_processed = dict()
for i, (key, value) in enumerate(metrics["forgettings"].items()):
    forgetting_processed[key] = [float("nan")] * i + value

fig = px.line(
    pd.DataFrame(forgetting_processed),
    title="Forgetting vs Task",
    labels=dict(index="Task", value="Forgetting"),
    width=800,
    height=600,
    markers=True,
)
fig.update_xaxes(type="category")
fig.show()

## Losses

In [ ]:
fig = px.line(
    pd.DataFrame(metrics["losses"]),
    title="Losses vs Task",
    labels=dict(index="Task", value="Loss"),
    width=800,
    height=600,
    markers=True,
)
fig.update_xaxes(type="category")
fig.show()

## CPU Usage

In [ ]:
fig = px.line(
    metrics["train_cpu_usage"],
    title="CPU Usage vs Batch",
    labels=dict(index="Batch", value="Value"),
    width=800,
    height=600,
)
fig.update_xaxes(type="category")
fig.show()

## Time per epoch

In [ ]:
fig = px.line(
    metrics["train_time_epoch"],
    title="Time per Epoch vs Epoch",
    labels=dict(index="Epoch", value="Value"),
    width=800,
    height=600,
)
fig.update_xaxes(type="category")
fig.show()

## Running time

In [ ]:
fig = px.line(
    metrics["train_running_time"],
    title="Running Time vs Batch",
    labels=dict(index="Batch", value="Value"),
    width=800,
    height=600,
)
fig.update_xaxes(type="category")
fig.show()

## Training loss

In [ ]:
fig = px.line(
    metrics["train_loss_epoch"],
    title="Loss vs Batch",
    labels=dict(index="Batch", value="Value"),
    width=800,
    height=600,
)
fig.update_xaxes(type="category")
fig.show()

## All components losses

In [ ]:
fig = px.line(
    metrics["all_loss_components"],
    title="Loss Components vs Batch",
    labels=dict(index="Batch", value="Value"),
    width=800,
    height=600,
)
fig.update_xaxes(type="category")
fig.show()

## Weights used

In [ ]:
used_weights = list(map(lambda x: sum((abs(x) > 1e-5)).item(), metrics["weights"]))

In [ ]:
fig = px.line(
    used_weights,
    title="Used Weights vs Task",
    labels=dict(index="Task", value="Value"),
    width=800,
    height=600,
)
fig.update_xaxes(type="category")
fig.show()

# Weight distribution

In [ ]:
weight_distributions = list(map(lambda x: sorted(x.flatten()), metrics["weights"]))

In [ ]:
for i in range(10):
    fig = px.histogram(
        weight_distributions[i],
        title=f"Weight Distribution for Task {i}",
        labels=dict(index="Weight", value="Value"),
        width=800,
        height=600,
    )
    fig.show()